In [1]:
!pip install numpy pandas scikit-learn joblib flask

In [2]:
import pandas as pd
df=pd.read_csv("/content/Symptom2Disease.csv")
df

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."
...,...,...,...
1195,295,diabetes,I'm shaking and trembling all over. I've lost ...
1196,296,diabetes,"Particularly in the crevices of my skin, I hav..."
1197,297,diabetes,I regularly experience these intense urges and...
1198,298,diabetes,"I have trouble breathing, especially outside. ..."


In [3]:
df.isnull().sum()

,0
Unnamed: 0,0
label,0
text,0


In [4]:
df.head()


,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."


In [5]:
df.tail()

,Unnamed: 0,label,text
1195,295,diabetes,I'm shaking and trembling all over. I've lost ...
1196,296,diabetes,"Particularly in the crevices of my skin, I hav..."
1197,297,diabetes,I regularly experience these intense urges and...
1198,298,diabetes,"I have trouble breathing, especially outside. ..."
1199,299,diabetes,I constantly sneeze and have a dry cough. My i...


In [6]:
df.describe()

,Unnamed: 0
count,1200.000000
mean,149.500000
std,86.638166
min,0.000000
25%,74.750000
50%,149.500000
75%,224.250000
max,299.000000


In [7]:
print(df['label'].value_counts())

label
Psoriasis                          50
Varicose Veins                     50
peptic ulcer disease               50
drug reaction                      50
gastroesophageal reflux disease    50
allergy                            50
urinary tract infection            50
Malaria                            50
Jaundice                           50
Cervical spondylosis               50
Migraine                           50
Hypertension                       50
Bronchial Asthma                   50
Acne                               50
Arthritis                          50
Dimorphic Hemorrhoids              50
Pneumonia                          50
Common Cold                        50
Fungal infection                   50
Dengue                             50
Impetigo                           50
Chicken pox                        50
Typhoid                            50
diabetes                           50
Name: count, dtype: int64


In [8]:
#label encoder
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
df['label']=lb.fit_transform(df['label'])

# Split into features (X) and target (y)
X = df['text']
y = df['label']

from sklearn.feature_extraction.text import TfidfVectorizer
# Convert text to numerical features using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(X).toarray()

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Test the model
y_pred = model.predict(X_test)

In [12]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=lb.classes_))

Accuracy: 0.9458333333333333
Classification Report:
                                  precision    recall  f1-score   support

                           Acne       1.00      1.00      1.00         7
                      Arthritis       1.00      1.00      1.00        10
               Bronchial Asthma       1.00      1.00      1.00        11
           Cervical spondylosis       1.00      1.00      1.00         7
                    Chicken pox       0.83      0.83      0.83        12
                    Common Cold       0.92      1.00      0.96        12
                         Dengue       1.00      0.67      0.80        12
          Dimorphic Hemorrhoids       1.00      1.00      1.00         7
               Fungal infection       1.00      1.00      1.00        13
                   Hypertension       1.00      1.00      1.00        10
                       Impetigo       1.00      1.00      1.00        11
                       Jaundice       1.00      1.00      1.00        

In [14]:
import joblib

# Save the model and vectorizer
joblib.dump(model, 'disease_prediction_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(lb, 'label_encoder.pkl')

['label_encoder.pkl']

In [17]:
# Load the model, vectorizer, and label encoder
model = joblib.load('disease_prediction_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# Test with a sample input
sample_symptom = "I have a runny nose and I am sneezing all the time."
sample_symptom_vectorized = vectorizer.transform([sample_symptom])
prediction = model.predict(sample_symptom_vectorized)
predicted_disease = label_encoder.inverse_transform(prediction)
print("Predicted Disease:", predicted_disease[0])

Predicted Disease: allergy


In [18]:
from google.colab import files
files.download('disease_prediction_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
!pip install flask pyngrok

In [5]:
!pip install flask pyngrok

from flask import Flask, request, jsonify
from pyngrok import ngrok
import joblib
import numpy as np
from sklearn.preprocessing import LabelEncoder

app = Flask(__name__)

# Load the trained model
model = joblib.load('disease_prediction_model.pkl')
label_encoder = joblib.load('label_encoder.pkl')  # Load label encoder

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json  # Get JSON data from request
        symptom = data['symptom']  # Extract symptom

        # Encode the symptom
        symptom_encoded = label_encoder.transform([symptom])

        # Predict disease
        prediction = model.predict([symptom_encoded])

        # Convert prediction back to disease name
        predicted_disease = label_encoder.inverse_transform(prediction)

        return jsonify({'predicted_disease': predicted_disease[0]})

    except Exception as e:
        return jsonify({'error': str(e)})

# **Get your authtoken from https://dashboard.ngrok.com/auth**
# **and replace 'YOUR_AUTHTOKEN' with your actual token**
ngrok.set_auth_token("YOUR_AUTHTOKEN")

ngrok_tunnel = ngrok.connect(5000)
print(f"Public URL: {ngrok_tunnel.public_url}")

app.run(port=5000)

ERROR:pyngrok.process.ngrok:t=2025-02-13T16:12:11+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-02-13T16:12:11+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: YOUR_AUTHTOKEN\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.